<a href="https://colab.research.google.com/github/zhan5555/MachineLearningProject/blob/main/Song_Recommendation_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook on NLP and Graph Neural Network Recommendation System Solution**

TPU Set Up
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tpu.ipynb#scrollTo=Mv7kehTZ1Lq_

In [ ]:
#  Install PyTorch and PyTorch Geometric Dependencies
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.4.1+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.4.1+cu121.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.4.1+cu121.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.4.1+cu121.html
!pip install torch-geometric
!pip install scikit-surprise
!pip install transformers
!pip install torch-xla
!pip install optuna # to use Bayesian Optimization on GNN model parameter search
!pip install joblib

Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3662068 sha256=5734b9f5aa05fe9a32ca9410248586e2ee1aede22bf82486c467613150104e71
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=2795429 sha256=5fb73016d00d666d77f329286cdbb79b5da4ff61bcdbebacd148674f8e617a5c
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built t

In [ ]:
!pip install torch-xla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 19.1 MB/s eta 0:00:00


In [ ]:
pip install optuna # to use Bayesian Optimization on GNN model parameter search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [ ]:
pip install joblib

In [ ]:
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import torch
import torch_geometric

In [ ]:
# Initialize the TPU strategy - intall packages before this cell
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)  # Use resolver directly

print("All devices: ", tf.config.list_logical_devices('TPU'))
print('TPU initialized successfully')

ValueError: Please provide a TPU Name to connect to.

In [ ]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

with tf.device('/TPU:0'):
  c = tf.matmul(a, b)

print("c device: ", c.device)
print(c)

c device:  /job:localhost/replica:0/task:0/device:TPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
# Run me 1 - Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run me 2
import warnings
warnings.filterwarnings('ignore')

# Basic libraries of python for numeric and dataframe computations
import numpy as np
import pandas as pd

# Basic library for data visualization
import matplotlib.pyplot as plt

# Slightly advanced library for data visualization
import seaborn as sns

# To compute the cosine similarity between two vectors
from sklearn.metrics.pairwise import cosine_similarity
# To compute the accuracy of models
from sklearn.metrics import accuracy_score

# A dictionary output that does not raise a key error
from collections import defaultdict

# A performance metrics in sklearn
from sklearn.metrics import mean_squared_error

# To compute the accuracy of models
from surprise import accuracy

# This class is used to parse a file containing play_counts, data should be in structure - user; item; play_count
from surprise.reader import Reader

# Class for loading datasets
from surprise.dataset import Dataset

# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV

# For splitting the data in train and test dataset
from surprise.model_selection import train_test_split

# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic

# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD

# For implementing KFold cross-validation
from surprise.model_selection import KFold

# For implementing clustering-based recommendation system
from surprise import CoClustering

import random
from surprise.model_selection import GridSearchCV, train_test_split
from surprise import Dataset, Reader, SVD, accuracy, KNNBasic
from surprise.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, davies_bouldin_score
from math import log2
from collections import defaultdict
import networkx as nx

In [ ]:
print(tf.__version__)  # Make sure TensorFlow is still the correct version
print(f"PyTorch version: {torch.__version__}")  # Verify Installation of PyTorch
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

2.17.0
PyTorch version: 2.5.0+cu121
PyTorch Geometric version: 2.6.1


In [ ]:
# Run me 3 - importing the datasets
count_df = pd.read_csv('/content/drive/MyDrive/Python Course/count_data.csv')
song_df = pd.read_csv('/content/drive/MyDrive/Python Course/song_data.csv')
# df_Bert = pd.read_csv('/content/drive/MyDrive/Python Course/df_Bert.csv')

## Create final df for text embedding

In [ ]:
# Run me 4 - create final df
# drop duplicated rows in sound_df. Then join sound_df to count_df on song ID and drop the column 'Unnamed: 0'. name new dataset df. print how many songs did not get joined to count df,print number rows and columns of df, print number of unique user IDs and song IDs from df, print top 5 rows

# Drop duplicate rows in song_df based on 'song_id'
song_df = song_df.drop_duplicates(subset=['song_id'])

# Merge song_df and count_df on 'song_id'
df = pd.merge(count_df, song_df, on='song_id', how='left')

# Drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in df.columns:
  df = df.drop('Unnamed: 0', axis=1)

# Calculate the number of songs that did not get joined
songs_not_joined = count_df.shape[0] - df.shape[0]
print(f"Number of songs that did not get joined to count_df: {songs_not_joined}")

# Print number of rows and columns of the merged DataFrame
print(f"Number of rows in df: {df.shape[0]}")
print(f"Number of columns in df: {df.shape[1]}")

# Print number of unique user IDs and song IDs from df
print(f"Number of unique user IDs: {df['user_id'].nunique()}")
print(f"Number of unique song IDs: {df['song_id'].nunique()}")

# Print the top 5 rows of df
print(df.head(5))

Number of songs that did not get joined to count_df: 0
Number of rows in df: 2000000
Number of columns in df: 7
Number of unique user IDs: 76353
Number of unique song IDs: 10000
                                    user_id             song_id  play_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995           1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B           2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0           1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D           1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273           1   

             title                        release    artist_name  year  
0         The Cove             Thicker Than Water   Jack Johnson     0  
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976  
2         Stronger                     Graduation     Kanye West  2007  
3   Constellations              In Between Dreams   Jac

In [ ]:
# Run me 5 -  Apply label encoding for "user_id" and "song_id"
# # Label encoding code

from sklearn.preprocessing import LabelEncoder

le_user = LabelEncoder()
le_song = LabelEncoder()

df['user_id'] = le_user.fit_transform(df['user_id'])
df['song_id'] = le_song.fit_transform(df['song_id'])

print(df.head())

   user_id  song_id  play_count            title  \
0    54961      153           1         The Cove   
1    54961      413           2  Entre Dos Aguas   
2    54961      736           1         Stronger   
3    54961      750           1   Constellations   
4    54961     1188           1     Learn To Fly   

                         release    artist_name  year  
0             Thicker Than Water   Jack Johnson     0  
1            Flamenco Para Niños  Paco De Lucia  1976  
2                     Graduation     Kanye West  2007  
3              In Between Dreams   Jack Johnson  2005  
4  There Is Nothing Left To Lose   Foo Fighters  1999  


In [ ]:
# Run me 6 - filter for FINAL DATASET

#We include users who have listened to at least 3 unique songs.
#Users have a total play count of 80 times or more.
#We include only songs that have an average user play count of over 2 times.

# Step 1: Filter songs that have an average play count of over 2 times
popular_songs = df.groupby('song_id')['play_count'].mean()
popular_songs = popular_songs[popular_songs > 2].index.tolist()
df_popular_songs = df[df['song_id'].isin(popular_songs)]

# Step 2: From the filtered songs, identify users who have total play counts of 80 or more and have listened to at least 3 unique songs
user_stats = df_popular_songs.groupby('user_id').agg({'play_count': 'sum', 'song_id': 'nunique'})
active_users = user_stats[(user_stats['play_count'] >= 80) & (user_stats['song_id'] >= 3)].index.tolist()
df_final = df_popular_songs[df_popular_songs['user_id'].isin(active_users)]


# Display info for the final DataFrame
print(f"Row count: {df_final.shape[0]}")
print(f"Column count: {df_final.shape[1]}")
print(f"Unique user IDs: {df_final['user_id'].nunique()}")
print(f"Unique song IDs: {df_final['song_id'].nunique()}")

Row count: 940730
Column count: 7
Unique user IDs: 20642
Unique song IDs: 7711


In [ ]:
# Ran me 8
# add a new column to update df final name "Popularity Score", for each song
# the formula is Unique Users Played per song ×(Unique Users Played per song / Total Play Count per song) and normalized with min and max calues. Display top 5 rows

# Assuming df_final is your DataFrame with columns: user_id, song_id, play_count

# Step 1: Calculate total play count per song
df_total_play_count = df_final.groupby('song_id')['play_count'].sum().reset_index()
df_total_play_count.rename(columns={'play_count': 'total_play_count'}, inplace=True)

# Step 2: Calculate the average play per user for each song
df_avg_play_per_user = df_final.groupby('song_id')['play_count'].mean().reset_index()
df_avg_play_per_user.rename(columns={'play_count': 'avg_play_per_user'}, inplace=True)

# Step 3: Merge total play count and average play per user into one DataFrame
df_song_popularity = pd.merge(df_total_play_count, df_avg_play_per_user, on='song_id')

# Step 4: Normalize both total play count and average play per user
df_song_popularity['total_play_count_norm'] = (df_song_popularity['total_play_count'] - df_song_popularity['total_play_count'].min()) / \
                                              (df_song_popularity['total_play_count'].max() - df_song_popularity['total_play_count'].min())

df_song_popularity['avg_play_per_user_norm'] = (df_song_popularity['avg_play_per_user'] - df_song_popularity['avg_play_per_user'].min()) / \
                                               (df_song_popularity['avg_play_per_user'].max() - df_song_popularity['avg_play_per_user'].min())

# Step 5: Calculate the final popularity score by combining the normalized values (you can adjust the weights)
df_song_popularity['popularity_score'] = (df_song_popularity['total_play_count_norm'] * 0.6) + \
                                         (df_song_popularity['avg_play_per_user_norm'] * 0.4)

# Step 6: Merge the popularity score back to the original DataFrame
df_final = pd.merge(df_final, df_song_popularity[['song_id', 'popularity_score']], on='song_id', how='left')

# df_final now has a new column 'popularity_score'
print(df_final.head())


   user_id  song_id  play_count                                         title  \
0    22570     1286           6                              Somebody To Love   
1    22570     1811          13  Ghosts 'n' Stuff (Original Instrumental Mix)   
2    22570     2409           6                           My Dad's Gone Crazy   
3    22570     2430           4                                   Missing You   
4    22570     3781           2                                 Ya Nada Queda   

                release           artist_name  year  popularity_score  
0             My Worlds         Justin Bieber  2010          0.134081  
1      Ghosts 'n' Stuff              Deadmau5  2009          0.094275  
2       The Eminem Show  Eminem / Hailie Jade  2006          0.034575  
3          Love To Love            John Waite  1984          0.084749  
4  Ultimate Hits Latino                 Kudai  2004          0.037017  


## Create Test Embedding DistilBERT

In [ ]:
# Make a copy of df_final
df_Bert = df_final.copy()

# Ensure all parts being concatenated are strings and create a new column 'text'
df_Bert['text'] = df_Bert['title'].astype(str) + " " + df_Bert['release'].astype(str) + " " + df_Bert['artist_name'].astype(str)

# Print the first few rows of df_Bert to check if the new column was added and the original columns were retained
print(df_Bert[['title', 'release', 'artist_name', 'text']].head())

# Optionally, print the column names to ensure no columns were dropped
print(df_Bert.columns)

                                          title               release  \
0                              Somebody To Love             My Worlds   
1  Ghosts 'n' Stuff (Original Instrumental Mix)      Ghosts 'n' Stuff   
2                           My Dad's Gone Crazy       The Eminem Show   
3                                   Missing You          Love To Love   
4                                 Ya Nada Queda  Ultimate Hits Latino   

            artist_name                                               text  
0         Justin Bieber           Somebody To Love My Worlds Justin Bieber  
1              Deadmau5  Ghosts 'n' Stuff (Original Instrumental Mix) G...  
2  Eminem / Hailie Jade  My Dad's Gone Crazy The Eminem Show Eminem / H...  
3            John Waite                Missing You Love To Love John Waite  
4                 Kudai           Ya Nada Queda Ultimate Hits Latino Kudai  
Index(['user_id', 'song_id', 'play_count', 'title', 'release', 'artist_name',
       'year', 'popul

### Run embeddings to text column, saving all columns and embeddings to dataframe [using GPU accelerator]

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf
import logging  # Import logging
import numpy as np
import pandas as pd
import pickle  # Import pickle for saving the DataFrame

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configure logging
logging.basicConfig(level=logging.INFO)

# Set TensorFlow to use GPU if available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available. Using GPU for embedding generation.")
else:
    print("GPU is not available. Check runtime settings.")

# Step 1: Set the default (float32) policy to avoid mixed precision conflicts
tf.keras.mixed_precision.set_global_policy('float32')

# Step 2: Load the DistilBERT tokenizer and model (using GPU)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# Ensure the model is used in inference mode (no training)
distilbert_model.trainable = False

# Step 3: Efficient function to generate DistilBERT embeddings on GPU
def generate_distilbert_embeddings(text_data, batch_size=64, max_len=35):
    """
    Generate DistilBERT embeddings for text data in batches using GPU.

    Args:
    - text_data: List of concatenated text strings.
    - batch_size: Number of texts to process per batch.
    - max_len: Maximum sequence length for DistilBERT input.

    Returns:
    - embeddings: Numpy array of DistilBERT embeddings for the text data.
    """
    embeddings = []

    # Process data in batches to manage memory
    for i in range(0, len(text_data), batch_size):
        logging.info(f"Processing batch {i // batch_size + 1} / {len(text_data) // batch_size + 1}...")

        # Get the current batch
        batch_texts = text_data[i:i + batch_size]

        # Tokenize text inputs
        inputs = tokenizer(batch_texts, return_tensors="tf", padding=True, truncation=True, max_length=max_len)

        # Run model inference with DistilBERT to get embeddings on GPU
        outputs = distilbert_model(inputs)

        # Extract [CLS] token embedding (the first token)
        batch_embeddings = outputs.last_hidden_state[:, 0, :]  # Keep as Tensor for now

        # Accumulate embeddings (convert to numpy)
        embeddings.append(batch_embeddings.numpy())

        logging.info(f"Finished processing batch {i // batch_size + 1}.")

        # Clear memory after each batch
        tf.keras.backend.clear_session()

        logging.info("Cleared memory after batch processing.")

    # Concatenate all batch embeddings
    return np.concatenate(embeddings, axis=0)

# Step 4: Run DistilBERT on the 'text' column in df_Bert using GPU
logging.info("Generating embeddings...")
text_data = df_Bert['text'].tolist()  # Convert the 'text' column to a list of strings
distilbert_embeddings = generate_distilbert_embeddings(text_data, batch_size=256, max_len=35)  # Adjust batch size as needed

# Check the output shape to ensure successful embedding generation
print(f"Generated DistilBERT embeddings shape: {distilbert_embeddings.shape}")

# Step 5: Add the generated embeddings to the original DataFrame
df_Bert['distilbert_embeddings'] = list(distilbert_embeddings)

# Step 6: Save the entire DataFrame (including embeddings) to Google Drive using Pickle
pickle_save_path = '/content/drive/MyDrive/Python Course/df_with_embeddings.pkl'
with open(pickle_save_path, 'wb') as f:
    pickle.dump(df_Bert, f)

print(f"DataFrame with embeddings saved to {pickle_save_path}.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU is available. Using GPU for embedding generation.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Generated DistilBERT embeddings shape: (940730, 768)
DataFrame with embeddings saved to /content/drive/MyDrive/Python Course/df_with_embeddings.pkl.


### deparacte- Saving all columns and embeddings to csv file

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertModel
import tensorflow as tf
import logging  # Import logging
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)

# Step 1: Initialize TPU strategy
# Step 2: Reset any precision policy to avoid mixed precision conflicts
# This ensures we use the default (float32) policy
tf.keras.mixed_precision.set_global_policy('float32')

# Step 3: Load the DistilBERT tokenizer and model within TPU strategy scope
with strategy.scope():
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Ensure the model is used in inference mode (no training)
    distilbert_model.trainable = False

# Step 4: Efficient function to generate DistilBERT embeddings on TPU
def generate_distilbert_embeddings(text_data, batch_size=64, max_len=35, from_pt=False):
    """
    Generate DistilBERT embeddings for text data in batches using TPU.

    Args:
    - text_data: List of concatenated text strings.
    - batch_size: Number of texts to process per batch.
    - max_len: Maximum sequence length for DistilBERT input.

    Returns:
    - embeddings: Numpy array of DistilBERT embeddings for the text data.
    """
    embeddings = []

    # Process data in batches to manage memory
    for i in range(0, len(text_data), batch_size):
        logging.info(f"Processing batch {i // batch_size + 1} / {len(text_data) // batch_size + 1}...")

        # Get the current batch
        batch_texts = text_data[i:i + batch_size]

        # Tokenize text inputs
        inputs = tokenizer(batch_texts, return_tensors="tf", padding=True, truncation=True, max_length=max_len)

        # Move computation inside the TPU strategy scope
        with strategy.scope():
            # Run model inference with DistilBERT to get embeddings
            outputs = distilbert_model(inputs)

            # Extract [CLS] token embedding (the first token)
            batch_embeddings = outputs.last_hidden_state[:, 0, :]  # Keep as Tensor for now

        # Accumulate embeddings (convert to numpy outside the TPU scope)
        embeddings.append(batch_embeddings.numpy())

        logging.info(f"Finished processing batch {i // batch_size + 1}.")

        # Clear memory after each batch
        tf.keras.backend.clear_session()

        logging.info("Cleared memory after batch processing.")

    # Concatenate all batch embeddings
    return np.concatenate(embeddings, axis=0)

# Step 5: Run DistilBERT on the 'text' column in df_Bert using TPU
logging.info("Generating embeddings...")
text_data = df_Bert['text'].tolist()  # Convert the 'text' column to a list of strings
distilbert_embeddings = generate_distilbert_embeddings(text_data, batch_size=256, max_len=35)  # Adjust batch size as needed

# Check the output shape to ensure successful embedding generation
print(f"Generated DistilBERT embeddings shape: {distilbert_embeddings.shape}")

np.save('distilbert_embeddings.npy', distilbert_embeddings)  # Save to file


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Generated DistilBERT embeddings shape: (940730, 768)


In [ ]:
# Load the saved embeddings
distilbert_embeddings = np.load('distilbert_embeddings.npy')

# Verify the shape to ensure it matches the number of rows in df_Bert
print(f"Loaded embeddings shape: {distilbert_embeddings.shape}")
print(f"DataFrame shape: {df_Bert.shape}")

# Add the embeddings as a new column in df_final_content
df_Bert['distilbert_embeddings'] = list(distilbert_embeddings)

df_Bert.head()

Loaded embeddings shape: (940730, 768)
DataFrame shape: (940730, 9)


,user_id,song_id,play_count,title,release,artist_name,year,popularity_score,text,distilbert_embeddings
0,22570,1286,6,Somebody To Love,My Worlds,Justin Bieber,2010,0.134081,Somebody To Love My Worlds Justin Bieber,"[-0.04428858, -0.11217374, 0.10928344, 0.02291..."
1,22570,1811,13,Ghosts 'n' Stuff (Original Instrumental Mix),Ghosts 'n' Stuff,Deadmau5,2009,0.094275,Ghosts 'n' Stuff (Original Instrumental Mix) G...,"[0.02514749, -0.38314658, 0.09424855, 0.083873..."
2,22570,2409,6,My Dad's Gone Crazy,The Eminem Show,Eminem / Hailie Jade,2006,0.034575,My Dad's Gone Crazy The Eminem Show Eminem / H...,"[0.069457486, -0.058404543, 0.0692633, 0.00753..."
3,22570,2430,4,Missing You,Love To Love,John Waite,1984,0.084749,Missing You Love To Love John Waite,"[-0.15310472, 0.0046700044, -0.13837491, -0.15..."
4,22570,3781,2,Ya Nada Queda,Ultimate Hits Latino,Kudai,2004,0.037017,Ya Nada Queda Ultimate Hits Latino Kudai,"[-0.33876556, -0.34565505, 0.35807294, 0.11415..."


In [ ]:
# Save df_Bert as a CSV file
csv_file_path = '/content/drive/MyDrive/Python Course/df_Bert.csv'  # Adjust the path if needed
df_Bert.to_csv(csv_file_path, index=False)

print(f"df_Bert saved to: {csv_file_path}")

df_Bert saved to: /content/drive/MyDrive/Python Course/df_Bert.csv


## Create GNN

**Graph Structure:**

The GNN needs to understand the relationship between users and songs. Each user and song can be treated as nodes in the graph, and the interactions (like play counts) can be treated as edges between nodes.
Node Features:

For the song nodes, you can use the distilbert_embeddings column (which contains the embeddings of the song metadata) as the node feature. This provides rich feature information for the songs in your graph.
For users, you might also need features, like user profile embeddings or other metadata, depending on what is available.

**Model Input:**

The distilbert_embeddings can be directly passed into the GNN as the feature vector for each song node.
The GNN will then use these embeddings (along with other features if provided) to propagate information through the graph and learn to predict the play count for each user-song pair.
Steps to Implement:


**Nodes**: You will have user and song nodes.
**Edges**: The edges will represent interactions between users and songs (e.g., a user listening to a song).
**Node Features:** The distilbert_embeddings will be used as the feature for the song nodes.

**Define GNN Model:**
You can use a GNN library like PyTorch Geometric or DGL to build your GNN model.
The distilbert_embeddings will be passed as input features for the song nodes.

In [ ]:
df_Bert.head()

,user_id,song_id,play_count,title,release,artist_name,year,popularity_score,text,distilbert_embeddings
0,22570,1286,6,Somebody To Love,My Worlds,Justin Bieber,2010,0.134081,Somebody To Love My Worlds Justin Bieber,[-4.42885794e-02 -1.12173744e-01 1.09283440e-...
1,22570,1811,13,Ghosts 'n' Stuff (Original Instrumental Mix),Ghosts 'n' Stuff,Deadmau5,2009,0.094275,Ghosts 'n' Stuff (Original Instrumental Mix) G...,[ 2.51474902e-02 -3.83146584e-01 9.42485482e-...
2,22570,2409,6,My Dad's Gone Crazy,The Eminem Show,Eminem / Hailie Jade,2006,0.034575,My Dad's Gone Crazy The Eminem Show Eminem / H...,[ 6.94574863e-02 -5.84045425e-02 6.92633018e-...
3,22570,2430,4,Missing You,Love To Love,John Waite,1984,0.084749,Missing You Love To Love John Waite,[-1.53104722e-01 4.67000436e-03 -1.38374910e-...
4,22570,3781,2,Ya Nada Queda,Ultimate Hits Latino,Kudai,2004,0.037017,Ya Nada Queda Ultimate Hits Latino Kudai,[-3.38765562e-01 -3.45655054e-01 3.58072937e-...


In [ ]:
# minimum and maxmium of popoularity score and play count
min_popularity = df_Bert['popularity_score'].min()
max_popularity = df_Bert['popularity_score'].max()
min_play_count = df_Bert['play_count'].min()
max_play_count = df_Bert['play_count'].max()

print(f"Minimum Popularity Score: {min_popularity}")
print(f"Maximum Popularity Score: {max_popularity}")
print(f"Minimum Play Count: {min_play_count}")
print(f"Maximum Play Count: {max_play_count}")

Minimum Popularity Score: 0.0002217513428275
Maximum Popularity Score: 0.7612439090521799
Minimum Play Count: 1
Maximum Play Count: 2213


In [ ]:
# prompt: check format of distilbert_embeddings
print(type(df_Bert['distilbert_embeddings'][0]))

<class 'str'>


In [ ]:
# Check if any rows are missing proper format
invalid_rows = df_Bert[df_Bert['distilbert_embeddings'].apply(lambda x: not (isinstance(x, str) and x.startswith('[') and x.endswith(']') and ',' in x))]

# Print out invalid rows
print(invalid_rows)

        user_id  song_id  play_count  \
0         22570     1286           6   
1         22570     1811          13   
2         22570     2409           6   
3         22570     2430           4   
4         22570     3781           2   
...         ...      ...         ...   
940725    64590     3660           2   
940726    64590     3736           4   
940727    64590     3744           3   
940728    64590     3893           1   
940729    64590     3963           1   

                                               title               release  \
0                                   Somebody To Love             My Worlds   
1       Ghosts 'n' Stuff (Original Instrumental Mix)      Ghosts 'n' Stuff   
2                                My Dad's Gone Crazy       The Eminem Show   
3                                        Missing You          Love To Love   
4                                      Ya Nada Queda  Ultimate Hits Latino   
...                                              ..

1. Popularity Score Scaling:
The popularity score is already between 0 and a little under 1, so further scaling might not be necessary. However, MinMax scaling will ensure it falls perfectly within a 0-1 range, which can help keep all features on a comparable scale.

Recommended Scaling: MinMax scaling from 0 to 1 (this will not drastically change the range).

2. Play Count Scaling:
The play count has a much wider range (1 to 2213), with high skewness (many small values, few large values). Applying a log transformation will compress this range and reduce skewness, making it easier for the GNN to learn from the data.

Recommended Scaling: Apply log transformation to compress the large range, followed by optional MinMax scaling to normalize between 0 and 1.

Create a simple 2-layer GNN Model with using GCNConv. The input is the concatenated song embeddings and normalized popularity score.

**Feature Representation Capacity:**

The hidden dimension size (128) controls the capacity of the network to learn feature representations. Higher values give the network more capacity to learn complex patterns but can lead to overfitting, especially with limited data.
In this case, the GCNConv layers map the input features (concatenated song embeddings + popularity score) to a latent space of 128 dimensions.
Flexibility and Tuning:

The hidden layer size (128 in this case) is a hyperparameter that can be tuned. You can experiment with different values (e.g., 64, 256, etc.) to see which one works best for your dataset and task.

**Trade-offs:**

Larger Sizes (e.g., 256, 512): Increase the model's ability to learn more complex representations, but require more computational resources and are more prone to overfitting.
Smaller Sizes (e.g., 64): Reduce model complexity and computational cost, but might not capture enough useful patterns from the input features.

run on CPU

Graph Neural Network (GNN): This is a broad category of neural networks designed to work with graph-structured data.
GCNConv: This is a particular kind of layer within a GNN. It stands for Graph Convolutional Network Convolution. It's one of the most common and fundamental building blocks for constructing GNNs.
What GCNConv Does

GCNConv performs a type of "message passing" where information is aggregated from neighboring nodes in the graph. Here's a simplified explanation:

Feature Gathering: For each node, the GCNConv layer gathers the feature vectors of its immediate neighbors.
Aggregation: It combines (aggregates) these neighbor features, often by averaging them.
Transformation: It applies a linear transformation (like a weight matrix multiplication) to the aggregated features.
Update: The node's own feature vector is updated based on this transformed information.
This process allows the GNN to learn representations of nodes that take into account the structure of the graph and the features of their neighbors.

How GCNConv Fits into a GNN Model

You typically use multiple GCNConv layers stacked together to create a deeper GNN model. This allows the network to learn more complex patterns and relationships within the graph.

### **Train a simple GNN model using 2 layers of GCNConv, 64 hidden layer, Learning rate set to 0.0001**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import re
import ast

# Option 1: Use 20% of the dataset (random sampling)
# Uncomment the following line if you want to use 20% of the dataset:
df_sampled = df_Bert.sample(frac=0.2, random_state=42)
#df_sampled = df_Bert  # Use the entire dataset without sampling

# Step 1: Log transform and normalize play count, and normalize popularity score
df_sampled['play_count_log'] = np.log1p(df_sampled['play_count'])  # log(1 + play_count)
scaler_play_count = MinMaxScaler()
df_sampled['play_count_scaled'] = scaler_play_count.fit_transform(df_sampled[['play_count_log']])

scaler_popularity = MinMaxScaler()
df_sampled['popularity_score_scaled'] = scaler_popularity.fit_transform(df_sampled[['popularity_score']])

# Step 2: Ensure distilbert_embeddings and other columns are processed
def fix_embedding_string(embedding_str):
    fixed_str = re.sub(r'(?<=\d)\s+(?=-?\d)', ',', embedding_str)
    return fixed_str

df_sampled['distilbert_embeddings'] = df_sampled['distilbert_embeddings'].apply(
    lambda x: np.array(ast.literal_eval(fix_embedding_string(x)), dtype=np.float32) if isinstance(x, str) else np.array(x, dtype=np.float32)
)

# Step 3: Prepare tensors for GNN input
user_ids = torch.tensor(df_sampled['user_id'].values, dtype=torch.long)
song_ids = torch.tensor(df_sampled['song_id'].values, dtype=torch.long)
play_counts = torch.tensor(df_sampled['play_count_scaled'].values, dtype=torch.float32)
popularity_scores = torch.tensor(df_sampled['popularity_score_scaled'].values, dtype=torch.float32).unsqueeze(1)

# Combine song embeddings with the scaled popularity score
song_embeddings = torch.tensor(np.stack(df_sampled['distilbert_embeddings'].values), dtype=torch.float32)
song_node_features = torch.cat([song_embeddings, popularity_scores], dim=1)

# Construct Graph Edges
edges = torch.stack([user_ids, song_ids], dim=0)

# Create Graph Data object for training
graph_data = Data(x=song_node_features, edge_index=edges)

# Step 4: Define Graph Neural Network (GNN)
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Step 5: Alternative Loss Function (Log-Cosh Loss)
def logcosh_loss(pred, target):
    return torch.mean(torch.log(torch.cosh(pred - target)))

# Step 6: Training the GNN model
# Initialize the model
hidden_layer_size = 64
model = GNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_layer_size, out_channels=1)

# Use CPU or GPU based on availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Optimizer with adjustable learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Learning rate set to 0.0001

# Move data to device
graph_data = graph_data.to(device)
play_counts = play_counts.to(device)

# Training loop
data_loader = DataLoader([graph_data], batch_size=8, shuffle=True)  # You can adjust batch size

for epoch in range(30):  # Increased number of epochs to 30
    model.train()

    for batch in data_loader:
        optimizer.zero_grad()

        # Forward pass
        out = model(batch.x, batch.edge_index)

        # Choose between MSE and Log-Cosh loss
        # train_loss = F.mse_loss(out.squeeze(), play_counts)  # MSE loss
        train_loss = logcosh_loss(out.squeeze(), play_counts)  # Log-Cosh loss

        # Backward pass and optimize
        train_loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {train_loss.item()}")

# Step 7: Make predictions on the entire dataset
model.eval()
with torch.no_grad():
    predicted_play_counts = model(graph_data.x, graph_data.edge_index).cpu().numpy()

# Step 8: Inverse scale the predicted play counts
predicted_play_counts_log = predicted_play_counts.reshape(-1, 1)  # Ensure predicted values are also 2D
predicted_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_play_counts_log))  # Reverse log scaling

# Actual play counts
actual_play_counts_log = play_counts.cpu().numpy().reshape(-1, 1)  # Reshape to 2D
actual_play_counts = np.expm1(scaler_play_count.inverse_transform(actual_play_counts_log))  # Reverse log scaling

# Step 9: Get predictions for specific users and songs
def get_prediction(user_id, song_id):
    matching_user_idx = (user_ids == user_id).nonzero(as_tuple=True)[0]
    matching_song_idx = (song_ids == song_id).nonzero(as_tuple=True)[0]

    if matching_user_idx.numel() == 0 or matching_song_idx.numel() == 0:
        print(f"No match found for user {user_id} and song {song_id}.")
        return None

    user_idx = matching_user_idx[0].item()
    return predicted_play_counts[user_idx]

# Display predictions for user 22570 and song 1286 (if in sample)
pred_22570_1286 = get_prediction(22570, 1286)
if pred_22570_1286 is not None:
    print(f"Predicted play count for user 22570 and song 1286: {pred_22570_1286}")

# Step 10: Calculate RMSE and MAE for the entire dataset
rmse = np.sqrt(mean_squared_error(actual_play_counts, predicted_play_counts))
mae = mean_absolute_error(actual_play_counts, predicted_play_counts)

print(f"RMSE on the entire dataset: {rmse}")
print(f"MAE on the entire dataset: {mae}")

# Step 11: Calculate Precision, Recall, and F1 Score for Top 30 Recommendations
def calculate_top_30_metrics(user_id, threshold=0.5, confidence_threshold=2.5):
    # Get predictions for the user
    user_song_idxs = (user_ids == user_id).nonzero(as_tuple=True)[0]

    # Sort predicted play counts for that user
    user_predicted_play_counts = predicted_play_counts[user_song_idxs]
    top_30_indices = np.argsort(-user_predicted_play_counts)[:30]  # Get top 30 indices

    # Actual play counts (relevant items) for this user
    user_actual_play_counts = actual_play_counts[user_song_idxs]

    # Calculate "relevant" predictions based on thresholds (convert to binary)
    relevant_predictions = (user_predicted_play_counts[top_30_indices] > confidence_threshold).astype(int)  # Convert to binary (0/1)
    relevant_actual = (user_actual_play_counts[top_30_indices] > threshold).astype(int)  # Convert to binary (0/1)

    # Ensure both arrays are 1D and of the same length
    relevant_predictions = relevant_predictions.flatten()
    relevant_actual = relevant_actual.flatten()

    # Calculate precision, recall, and f1 score
    precision = precision_score(relevant_actual, relevant_predictions, zero_division=0)
    recall = recall_score(relevant_actual, relevant_predictions, zero_division=0)
    f1 = f1_score(relevant_actual, relevant_predictions, zero_division=0)

    return precision, recall, f1


# Example for user 22570
precision_22570, recall_22570, f1_22570 = calculate_top_30_metrics(22570)
print(f"Precision for user 22570: {precision_22570}")
print(f"Recall for user 22570: {recall_22570}")
print(f"F1 score for user 22570: {f1_22570}")


Epoch 1, Loss: 0.17438112199306488
Epoch 2, Loss: 0.14012163877487183
Epoch 3, Loss: 0.10928932577371597
Epoch 4, Loss: 0.08229158073663712
Epoch 5, Loss: 0.059502746909856796
Epoch 6, Loss: 0.041335225105285645
Epoch 7, Loss: 0.02814849279820919
Epoch 8, Loss: 0.02021971344947815
Epoch 9, Loss: 0.017533522099256516
Epoch 10, Loss: 0.01946721039712429
Epoch 11, Loss: 0.024339616298675537
Epoch 12, Loss: 0.029787564650177956
Epoch 13, Loss: 0.03398742154240608
Epoch 14, Loss: 0.03609348088502884
Epoch 15, Loss: 0.03601711243391037
Epoch 16, Loss: 0.03411194682121277
Epoch 17, Loss: 0.030944067984819412
Epoch 18, Loss: 0.027147794142365456
Epoch 19, Loss: 0.023325225338339806
Epoch 20, Loss: 0.01999030075967312
Epoch 21, Loss: 0.01750047318637371
Epoch 22, Loss: 0.016005517914891243
Epoch 23, Loss: 0.015442807227373123
Epoch 24, Loss: 0.015609272755682468
Epoch 25, Loss: 0.016234857961535454
Epoch 26, Loss: 0.017048152163624763
Epoch 27, Loss: 0.017814552411437035
Epoch 28, Loss: 0.01836

### Hyperparameter tuning

search on full dataset size

In [ ]:
print(torch.cuda.is_available())  # Should return True if GPU is enabled
print(torch.cuda.get_device_name(0))  # To display the name of the GPU

True
Tesla T4


### Optuna's Bayesian optimization hyperparamater search using data frame saved in Pickle

Best MAE:  4.262037551767267
Best Hyperparameters:  {'learning_rate': 8.032781689976049e-05, 'hidden_size': 128, 'epochs': 11}

In [ ]:
import pickle
import torch
import optuna
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Load the DataFrame from the Pickle file
with open('/content/drive/MyDrive/Python Course/df_with_embeddings.pkl', 'rb') as f:
    df_Bert = pickle.load(f)

# Check if GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Apply log transformation to play counts
df_Bert['play_count_log'] = np.log1p(df_Bert['play_count'])  # Apply log transformation

# Apply scaling to other features
scaler_play_count = MinMaxScaler()  # Initialize scaler
df_Bert['play_count_scaled'] = scaler_play_count.fit_transform(df_Bert[['play_count_log']])

# Convert the 'distilbert_embeddings' column from lists (loaded from Pickle) to tensors
song_node_features = torch.tensor(np.stack(df_Bert['distilbert_embeddings']), dtype=torch.float32).to(device)
play_counts = torch.tensor(df_Bert['play_count_scaled'].values, dtype=torch.float32).to(device)

# Weights to handle skewness in the play counts
weights = 1 / np.log1p(df_Bert['play_count'])  # Inverse of the log to give more weight to lower play counts

# Step 2: GNN Model definition with deeper architecture and dropout for regularization
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(GNNModel, self).__init__()
        self.layer1 = torch.nn.Linear(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer3 = torch.nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x = F.relu(self.bn1(self.layer1(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.relu(self.bn2(self.layer2(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = self.layer3(x)
        return x

# Xavier weight initialization
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

# Loss function with weighting based on skewness
def weighted_loss(predictions, targets, weights):
    loss = F.mse_loss(predictions, targets, reduction='none')
    weighted_loss = (weights * loss).mean()  # Apply weights
    return weighted_loss

# Step 3: Objective function for Optuna hyperparameter search
def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128])
    num_epochs = trial.suggest_int('epochs', 10, 50)

    # Initialize model with optimized architecture
    model = GNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_size, out_channels=1, dropout=0.3)
    model.apply(init_weights)  # Apply weight initialization
    model = model.to(device)  # Move model to GPU

    optimizer = Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(song_node_features, None)

        # Check for NaN/Inf in output
        if torch.isinf(out).any() or torch.isnan(out).any():
            print("NaN or Inf found in model output!")
            return float('inf')  # Skip this trial if invalid output

        # Calculate weighted loss
        train_loss = weighted_loss(out.squeeze(), play_counts, torch.tensor(weights, dtype=torch.float32).to(device))

        if torch.isnan(train_loss) or torch.isinf(train_loss):
            print("NaN or Inf found in training loss!")
            return float('inf')  # Skip this trial if invalid loss

        # Clip gradients to avoid instability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        train_loss.backward()
        optimizer.step()  # Perform optimizer step on GPU

    # Evaluation
    model.eval()
    with torch.no_grad():
        predicted_play_counts = model(song_node_features, None).cpu().numpy()  # Move output back to CPU for processing

        # Re-transform predicted play counts back to original scale
        predicted_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_play_counts.reshape(-1, 1)))

        # Check for Inf values after applying transformations
        if np.isinf(predicted_play_counts).any() or np.isnan(predicted_play_counts).any():
            print("Invalid values detected in predictions after transformation!")
            return float('inf')  # Skip this trial if invalid predictions

    # Calculate MAE and RMSE
    mae = mean_absolute_error(df_Bert['play_count'].values, predicted_play_counts)
    rmse = np.sqrt(mean_squared_error(df_Bert['play_count'].values, predicted_play_counts))

    print(f"Trial {trial.number}: MAE: {mae}, RMSE: {rmse}")
    return mae  # Optimize for MAE

# Step 4: Run Optuna with parallelization and GPU
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, n_jobs=1)  # Run with GPU, adjust n_jobs as needed

# Step 5: Print the best result
print("Best MAE: ", study.best_value)
print("Best Hyperparameters: ", study.best_params)


Using device: cuda


[I 2024-10-21 00:35:38,785] A new study created in memory with name: no-name-c07e1b11-27f1-48c3-b989-7587f350bfa2
[I 2024-10-21 00:35:46,275] Trial 0 finished with value: 32.87287403933503 and parameters: {'learning_rate': 3.630647784776808e-06, 'hidden_size': 64, 'epochs': 27}. Best is trial 0 with value: 32.87287403933503.


Trial 0: MAE: 32.87287403933503, RMSE: 115.5219714096843


[I 2024-10-21 00:35:59,929] Trial 1 finished with value: 6163.131255290593 and parameters: {'learning_rate': 7.459742265851282e-06, 'hidden_size': 128, 'epochs': 38}. Best is trial 0 with value: 32.87287403933503.


Trial 1: MAE: 6163.131255290593, RMSE: 568313.4202824809


...

Trial 197: MAE: 9.124648859715172, RMSE: 19.919677652106014


[I 2024-10-21 00:52:13,122] Trial 198 finished with value: 5.561966030530591 and parameters: {'learning_rate': 7.483424372905987e-05, 'hidden_size': 64, 'epochs': 12}. Best is trial 52 with value: 4.262037551767267.


Trial 198: MAE: 5.561966030530591, RMSE: 12.172381875434104


[I 2024-10-21 00:52:15,637] Trial 199 finished with value: 4.519018509431757 and parameters: {'learning_rate': 6.667760226873097e-05, 'hidden_size': 64, 'epochs': 10}. Best is trial 52 with value: 4.262037551767267.


Trial 199: MAE: 4.519018509431757, RMSE: 11.625528928994955
Best MAE:  4.262037551767267
Best Hyperparameters:  {'learning_rate': 8.032781689976049e-05, 'hidden_size': 128, 'epochs': 11}


Adjust range for another search:
changed activation function from ReLU to Leaky ReLU to handle skewed and sparse data better.

Changed search space to: (lr= suggest_loguniform explore  for a continous range between a minimum and maxmium)
    lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 512])
    num_epochs = trial.suggest_int('epochs', 10, 50)

In [ ]:
import pickle
import torch
import optuna
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Load the DataFrame from the Pickle file
with open('/content/drive/MyDrive/Python Course/df_with_embeddings.pkl', 'rb') as f:
    df_Bert = pickle.load(f)

# Check if GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Apply log transformation to play counts
df_Bert['play_count_log'] = np.log1p(df_Bert['play_count'])  # Apply log transformation

# Apply scaling to play counts
scaler_play_count = MinMaxScaler()  # Initialize scaler
df_Bert['play_count_scaled'] = scaler_play_count.fit_transform(df_Bert[['play_count_log']])

# Use CPU for song embeddings to avoid GPU memory overload
song_node_features = torch.tensor(np.stack(df_Bert['distilbert_embeddings']), dtype=torch.float32)  # Stays on CPU

# Play counts will be moved to GPU during training
play_counts = torch.tensor(df_Bert['play_count_scaled'].values, dtype=torch.float32).to(device)

# Create DataLoader for mini-batching
dataset = TensorDataset(song_node_features, play_counts)
batch_size = 64  # Adjust based on available memory
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Weights to handle skewness in the play counts (based on inverse log)
weights = torch.tensor(1 / np.log1p(df_Bert['play_count'].values), dtype=torch.float32).to(device)

# Step 2: GNN Model definition with deeper architecture and dropout for regularization
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(GNNModel, self).__init__()
        self.layer1 = torch.nn.Linear(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer3 = torch.nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, edge_index=None):
        x = F.leaky_relu(self.bn1(self.layer1(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.layer2(x)))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

# Xavier weight initialization
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

# Loss function with weighting based on skewness
def weighted_loss(predictions, targets, weights):
    loss = F.mse_loss(predictions, targets, reduction='none')
    weighted_loss = (weights * loss).mean()  # Apply weights
    return weighted_loss

# Step 3: Objective function for Optuna hyperparameter search
def objective(trial):
    # Clear GPU cache before each trial
    torch.cuda.empty_cache()

    # Hyperparameter search space
    lr = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3])
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 512])
    num_epochs = trial.suggest_int('epochs', 10, 50)

    # Initialize model with optimized architecture
    model = GNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_size, out_channels=1, dropout=0.3)
    model.apply(init_weights)  # Apply weight initialization
    model = model.to(device)  # Move model to GPU

    optimizer = Adam(model.parameters(), lr=lr)

    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_features, batch_labels in dataloader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            optimizer.zero_grad()

            # Mixed precision forward pass
            with torch.cuda.amp.autocast():
                out = model(batch_features)
                train_loss = weighted_loss(out.squeeze(), batch_labels, weights)

            # Check for NaN/Inf in output
            if torch.isnan(train_loss) or torch.isinf(train_loss):
                print("NaN or Inf found in training loss!")
                return float('inf')  # Skip this trial if invalid loss

            # Mixed precision backward pass
            scaler.scale(train_loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += train_loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        predicted_play_counts = model(song_node_features.to(device), None).cpu().numpy()

        # Re-transform predicted play counts back to original scale
        predicted_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_play_counts.reshape(-1, 1)))

        # Check for Inf values after applying transformations
        if np.isinf(predicted_play_counts).any() or np.isnan(predicted_play_counts).any():
            print("Invalid values detected in predictions after transformation!")
            return float('inf')  # Skip this trial if invalid predictions

    # Calculate MAE and RMSE
    mae = mean_absolute_error(df_Bert['play_count'].values, predicted_play_counts)
    rmse = np.sqrt(mean_squared_error(df_Bert['play_count'].values, predicted_play_counts))

    print(f"Trial {trial.number}: MAE: {mae}, RMSE: {rmse}")
    return mae  # Optimize for MAE

# Step 4: Run Optuna with parallelization and GPU
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, n_jobs=1)  # Run with GPU, adjust n_jobs as needed

# Step 5: Print the best result
print("Best MAE: ", study.best_value)
print("Best Hyperparameters: ", study.best_params)

ModuleNotFoundError: No module named 'optuna'

### Grid search with 20% sample of dataset

In [ ]:
import optuna
import torch
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assume df_sampled is your preprocessed DataFrame

# Step 1: Apply log transformation to play counts to reduce skewness
df_sampled['play_count_log'] = np.log1p(df_sampled['play_count'])  # Apply log transformation

# Step 2: Apply scaling to other features, if necessary
scaler_play_count = MinMaxScaler()  # Initialize scaler
df_sampled['play_count_scaled'] = scaler_play_count.fit_transform(df_sampled[['play_count_log']])

# Ensure embeddings and features are already in df_sampled, e.g., distilbert_embeddings
song_node_features = torch.tensor(np.stack(df_sampled['distilbert_embeddings']), dtype=torch.float32)
play_counts = torch.tensor(df_sampled['play_count_scaled'].values, dtype=torch.float32)

# Weights to handle skewness in the play counts
weights = 1 / np.log1p(df_sampled['play_count'])  # Inverse of the log to give more weight to lower play counts

# Step 3: GNN Model definition with deeper architecture and dropout for regularization
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(GNNModel, self).__init__()
        self.layer1 = torch.nn.Linear(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.layer3 = torch.nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x = F.relu(self.bn1(self.layer1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.layer2(x)))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

# Xavier weight initialization
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

# Loss function with weighting based on skewness
def weighted_loss(predictions, targets, weights):
    loss = F.mse_loss(predictions, targets, reduction='none')
    weighted_loss = (weights * loss).mean()  # Apply weights
    return weighted_loss

# Step 4: Objective function for Optuna hyperparameter search
def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_loguniform('learning_rate', 1e-6, 1e-4)
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128])
    num_epochs = trial.suggest_int('epochs', 10, 50)

    # Initialize model with optimized architecture
    model = GNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_size, out_channels=1, dropout=0.3)
    model.apply(init_weights)  # Apply weight initialization
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(song_node_features, None)  # Replace None with edge_index if using graph data

        # Check for NaN/Inf in output
        if torch.isinf(out).any() or torch.isnan(out).any():
            print("NaN or Inf found in model output!")
            return float('inf')  # Skip this trial if invalid output

        # Calculate weighted loss
        train_loss = weighted_loss(out.squeeze(), play_counts, torch.tensor(weights, dtype=torch.float32))

        if torch.isnan(train_loss) or torch.isinf(train_loss):
            print("NaN or Inf found in training loss!")
            return float('inf')  # Skip this trial if invalid loss

        # Clip gradients to avoid instability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        train_loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        predicted_play_counts = model(song_node_features, None).cpu().numpy()

        # Re-transform predicted play counts back to original scale
        predicted_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_play_counts.reshape(-1, 1)))

        # Check for Inf values after applying transformations
        if np.isinf(predicted_play_counts).any() or np.isnan(predicted_play_counts).any():
            print("Invalid values detected in predictions after transformation!")
            return float('inf')  # Skip this trial if invalid predictions

    # Calculate MAE and RMSE
    mae = mean_absolute_error(df_sampled['play_count'].values, predicted_play_counts)
    rmse = np.sqrt(mean_squared_error(df_sampled['play_count'].values, predicted_play_counts))

    print(f"Trial {trial.number}: MAE: {mae}, RMSE: {rmse}")
    return mae  # Optimize for MAE

# Step 5: Run Optuna with parallelization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, n_jobs=90)  # Adjust n_trials and n_jobs based on system resources

# Step 6: Print the best result
print("Best MAE: ", study.best_value)
print("Best Hyperparameters: ", study.best_params)


[I 2024-10-20 02:11:05,106] A new study created in memory with name: no-name-cb7ad879-aaee-4cea-b583-9a93ce5e2649
[I 2024-10-20 02:15:14,105] Trial 7 finished with value: 5.047344842321312 and parameters: {'learning_rate': 2.4733774792465053e-06, 'hidden_size': 64, 'epochs': 10}. Best is trial 7 with value: 5.047344842321312.


Trial 7: MAE: 5.047344842321312, RMSE: 10.155525226457222


[I 2024-10-20 02:15:15,738] Trial 51 finished with value: 5.59976945900216 and parameters: {'learning_rate': 5.336759111020201e-06, 'hidden_size': 64, 'epochs': 10}. Best is trial 7 with value: 5.047344842321312.


Trial 51: MAE: 5.59976945900216, RMSE: 11.94689452216554


[I 2024-10-20 02:15:56,947] Trial 43 finished with value: 4.360305675380037 and parameters: {'learning_rate': 3.904512741480352e-06, 'hidden_size': 64, 'epochs': 12}. Best is trial 43 with value: 4.360305675380037.


...

Trial 150: MAE: 31.135138623332498, RMSE: 584.7191814692341


[I 2024-10-20 02:42:48,145] Trial 113 finished with value: 124.26807074715627 and parameters: {'learning_rate': 2.19093436512288e-06, 'hidden_size': 128, 'epochs': 44}. Best is trial 179 with value: 4.037038643174347.


Trial 113: MAE: 124.26807074715627, RMSE: 4102.584581561533


[I 2024-10-20 02:42:49,557] Trial 111 finished with value: 1570.8048195133003 and parameters: {'learning_rate': 2.1370473077401712e-05, 'hidden_size': 128, 'epochs': 47}. Best is trial 179 with value: 4.037038643174347.


Trial 111: MAE: 1570.8048195133003, RMSE: 43479.033006188576
Best MAE:  4.037038643174347
Best Hyperparameters:  {'learning_rate': 9.425682782351999e-05, 'hidden_size': 64, 'epochs': 11}


In [ ]:
import optuna
import torch
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

epsilon = 1e-10  # Small value to avoid log of zero or negative numbers

# Apply safe log transformation to play counts
df_sampled['play_count_log'] = np.log1p(df_sampled['play_count'] + epsilon)

# Function to check for NaNs or Inf values
def check_for_invalid_values(data, name="Data"):
    """
    Check for NaNs or Inf values in the data and print a warning if found.

    Args:
    - data: The data to check (NumPy array or PyTorch tensor).
    - name: A name for the data to print in the warning (string).

    Returns:
    - True if invalid values are found, False otherwise.
    """
    if torch.is_tensor(data):  # If data is a PyTorch tensor
        has_nan = torch.isnan(data).sum().item() > 0
        has_inf = torch.isinf(data).sum().item() > 0
    else:  # Assume it's a NumPy array or pandas DataFrame
        has_nan = np.isnan(data).sum() > 0
        has_inf = np.isinf(data).sum() > 0

    if has_nan or has_inf:
        print(f"Invalid values detected in {name}: NaN count = {has_nan}, Inf count = {has_inf}")
        return True
    return False

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    num_epochs = trial.suggest_int('epochs', 10, 30)

    # Initialize model and optimizer
    model = GNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_size, out_channels=1)
    model = model.to(device)
    optimizer = Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(graph_data.x, graph_data.edge_index)

        # Check for NaN/Inf in output
        if torch.isinf(out).any() or torch.isnan(out).any():
            print("NaN or Inf found in model output!")
            return float('inf')  # Skip this trial if invalid output

        train_loss = logcosh_loss(out.squeeze(), play_counts)

        if torch.isnan(train_loss) or torch.isinf(train_loss):
            print("NaN or Inf found in training loss!")
            return float('inf')  # Skip this trial if invalid loss

        train_loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        predicted_play_counts = model(graph_data.x, graph_data.edge_index).cpu().numpy()

        # Check for Inf values in predictions
        if np.isinf(predicted_play_counts).any():
            print("Inf found in predicted play counts! Replacing with large finite values.")
            predicted_play_counts = np.nan_to_num(predicted_play_counts, nan=0.0, posinf=np.finfo(np.float32).max)

        predicted_play_counts_log = predicted_play_counts.reshape(-1, 1)
        predicted_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_play_counts_log))

    # Check for Inf values after applying transformations
    if np.isinf(predicted_play_counts).any() or np.isnan(predicted_play_counts).any():
        print("Invalid values detected in predictions after transformation!")
        return float('inf')  # Skip this trial if invalid predictions

    # Calculate MAE and RMSE
    mae = mean_absolute_error(actual_play_counts, predicted_play_counts)
    rmse = np.sqrt(mean_squared_error(actual_play_counts, predicted_play_counts))

    print(f"Trial {trial.number}: MAE: {mae}, RMSE: {rmse}")
    return mae  # Optimize for MAE

# Check for NaNs and Infs in the dataset before starting optimization
if check_for_invalid_values(graph_data.x, "Input features") or check_for_invalid_values(play_counts, "Play counts"):
    print("Invalid values detected in input data!")
else:
    # Set the number of trials and jobs
    n_trials = 200  # Number of trials you want
    n_jobs = 90     # Set to 90 to utilize the available cores

    # Set up the study
    study = optuna.create_study(direction='minimize')

    # Run the optimization directly using Optuna's built-in parallelization
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)  # Optuna will handle the parallel execution

    # After all trials are complete, print the best results
    print("Best MAE: ", study.best_value)
    print("Best Hyperparameters: ", study.best_params)


[I 2024-10-20 01:51:59,164] A new study created in memory with name: no-name-365e6232-b16a-43ac-801b-dbe1417e3fe6
[I 2024-10-20 01:52:10,233] Trial 24 finished with value: inf and parameters: {'learning_rate': 0.01657081209327333, 'hidden_size': 32, 'epochs': 15}. Best is trial 24 with value: inf.


NaN or Inf found in training loss!


[I 2024-10-20 01:52:16,796] Trial 49 finished with value: inf and parameters: {'learning_rate': 0.018540201968360353, 'hidden_size': 32, 'epochs': 15}. Best is trial 24 with value: inf.


NaN or Inf found in training loss!
NaN or Inf found in training loss!
Trial 9: MAE: 180.56639099121094, RMSE: 36664.38671875


[I 2024-10-20 01:52:19,946] Trial 9 finished with value: 180.56639099121094 and parameters: {'learning_rate': 0.0004383709507550346, 'hidden_size': 32, 'epochs': 11}. Best is trial 9 with value: 180.56639099121094.
[I 2024-10-20 01:52:20,013] Trial 11 finished with value: inf and parameters: {'learning_rate': 0.04288127529457909, 'hidden_size': 64, 'epochs': 10}. Best is trial 9 with value: 180.56639099121094.


...

Trial 106: MAE: 4.0410685539245605, RMSE: 35.922508239746094


[I 2024-10-20 01:59:49,424] Trial 192 finished with value: 4.4003705978393555 and parameters: {'learning_rate': 0.0027882216701632127, 'hidden_size': 128, 'epochs': 25}. Best is trial 19 with value: 3.170053005218506.
[I 2024-10-20 01:59:49,482] Trial 105 finished with value: 5.958810806274414 and parameters: {'learning_rate': 0.0001262135871040775, 'hidden_size': 128, 'epochs': 30}. Best is trial 19 with value: 3.170053005218506.
[I 2024-10-20 01:59:49,495] Trial 109 finished with value: 3019.1474609375 and parameters: {'learning_rate': 0.00010750305731177661, 'hidden_size': 128, 'epochs': 30}. Best is trial 19 with value: 3.170053005218506.


Trial 192: MAE: 4.4003705978393555, RMSE: 9.85139274597168
Trial 105: MAE: 5.958810806274414, RMSE: 244.8497314453125
Trial 109: MAE: 3019.1474609375, RMSE: 693882.4375


[I 2024-10-20 01:59:50,113] Trial 193 finished with value: 4.163294792175293 and parameters: {'learning_rate': 0.0028763794958448566, 'hidden_size': 128, 'epochs': 26}. Best is trial 19 with value: 3.170053005218506.
[I 2024-10-20 01:59:50,126] Trial 190 finished with value: 3.385586977005005 and parameters: {'learning_rate': 0.0030237941921029206, 'hidden_size': 128, 'epochs': 26}. Best is trial 19 with value: 3.170053005218506.


Trial 193: MAE: 4.163294792175293, RMSE: 9.746049880981445
Trial 190: MAE: 3.385586977005005, RMSE: 9.329413414001465
Best MAE:  3.170053005218506
Best Hyperparameters:  {'learning_rate': 0.005293590075732399, 'hidden_size': 32, 'epochs': 27}


In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

96


### Run on TPU- keep crashing

In [ ]:
import torch_xla.core.xla_model as xm
import re
import ast
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.loader import DataLoader
import numpy as np

# Step 1: Log transform and normalize play count, and normalize popularity score
df_Bert['play_count_log'] = np.log1p(df_Bert['play_count'])  # log(1 + play_count)
scaler_play_count = MinMaxScaler()
df_Bert['play_count_scaled'] = scaler_play_count.fit_transform(df_Bert[['play_count_log']])

scaler_popularity = MinMaxScaler()
df_Bert['popularity_score_scaled'] = scaler_popularity.fit_transform(df_Bert[['popularity_score']])

# Step 2: Randomly sample 20% of the dataset
df_sampled = df_Bert.sample(frac=0.2, random_state=42)  # 20% of the data

# Step 3: Fix distilbert_embeddings string formatting (if necessary)
def fix_embedding_string(embedding_str):
    fixed_str = re.sub(r'(?<=\d)\s+(?=-?\d)', ',', embedding_str)
    return fixed_str

# Step 4: Parse distilbert_embeddings if they are strings
df_sampled['distilbert_embeddings'] = df_sampled['distilbert_embeddings'].apply(
    lambda x: np.array(ast.literal_eval(fix_embedding_string(x)), dtype=np.float32) if isinstance(x, str) else np.array(x, dtype=np.float32)
)

# Step 5: Convert distilbert_embeddings to a tensor with float32 dtype
song_embeddings = torch.tensor(np.stack(df_sampled['distilbert_embeddings'].values), dtype=torch.float32)

# Step 6: Prepare tensors for GNN input with consistent float32 dtype
user_ids = torch.tensor(df_sampled['user_id'].values, dtype=torch.long)
song_ids = torch.tensor(df_sampled['song_id'].values, dtype=torch.long)
play_counts = torch.tensor(df_sampled['play_count_scaled'].values, dtype=torch.float32)  # Scaled play count in float32
popularity_scores = torch.tensor(df_sampled['popularity_score_scaled'].values, dtype=torch.float32).unsqueeze(1)

# Combine song embeddings with the scaled popularity score
song_node_features = torch.cat([song_embeddings, popularity_scores], dim=1)

# Step 7: Define a Simplified Graph Neural Network (GNN) with reduced layers
class SimpleGNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SimpleGNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Reduce hidden layer size to avoid memory overload
hidden_layer_size = 32  # Further reduce size for testing

# Step 8: Construct Graph Edges
edges = torch.stack([user_ids, song_ids], dim=0)

# Step 9: Use a Single TPU Core for Debugging
device = xm.xla_device()  # Use single TPU core for testing

# Move the data to TPU (subset)
song_node_features = song_node_features.to(device)
edges = edges.to(device)
play_counts = play_counts.to(device)

# Create graph data (subset)
graph_data = Data(x=song_node_features, edge_index=edges).to(device)

# Move the model to the TPU (simplified model)
model = SimpleGNNModel(in_channels=song_node_features.shape[1], hidden_channels=hidden_layer_size, out_channels=1).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 10: Train with Mini-Batches (Sampled Data)
data_loader = DataLoader([graph_data], batch_size=8, shuffle=True)  # Further reduce batch size for testing

# Training loop (reduce epochs for testing)
for epoch in range(10):
    model.train()

    for batch in data_loader:
        optimizer.zero_grad()

        # Run GNN on the graph batch
        out = model(batch.x, batch.edge_index)

        # Compute the loss for the training set (subset)
        train_loss = F.mse_loss(out, play_counts)

        train_loss.backward()
        xm.optimizer_step(optimizer)  # Perform optimizer step on TPU

    print(f"Epoch {epoch+1}, Loss: {train_loss.item()}")

# Step 11: Evaluate predictions
model.eval()
with torch.no_grad():
    predictions = model(graph_data.x, graph_data.edge_index)

# Step 12: Inverse scale the predicted play counts
predicted_play_counts = scaler_play_count.inverse_transform(predictions.cpu().numpy())  # Reverse scaling

# Step 13: Get predictions for specific users and songs (subset)
def get_prediction(user_id, song_id):
    user_idx = (user_ids == user_id).nonzero(as_tuple=True)[0].item()
    song_idx = (song_ids == song_id).nonzero(as_tuple=True)[0].item()
    return predicted_play_counts[user_idx]

# Display predictions for user 22570 and song 1286 (if in sample)
pred_22570_1286 = get_prediction(22570, 1286)
print(f"Predicted play count for user 22570 and song 1286: {pred_22570_1286}")

### Train model with optimized paramaters

In [ ]:
import pickle
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.optim import Adam
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Step 2: Load the DataFrame with embeddings from Pickle file
with open('/content/drive/MyDrive/Python Course/df_with_embeddings.pkl', 'rb') as f:
    df_Bert = pickle.load(f)

# Step 3: Set device to GPU (or CPU if GPU is unavailable)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 4: Apply log transformation and scaling to play counts
df_Bert['play_count_log'] = np.log1p(df_Bert['play_count'])
scaler_play_count = MinMaxScaler()
df_Bert['play_count_scaled'] = scaler_play_count.fit_transform(df_Bert[['play_count_log']])

# Step 5: Split the dataset into training and test sets (80% train, 20% test)
train_df, test_df = train_test_split(df_Bert, test_size=0.2, random_state=42)

# Step 6: Convert embeddings and play counts to tensors and move to GPU (for training and testing sets)
train_embeddings = torch.tensor(np.stack(train_df['distilbert_embeddings']), dtype=torch.float32).to(device)
train_play_counts = torch.tensor(train_df['play_count_scaled'].values, dtype=torch.float32).to(device)

test_embeddings = torch.tensor(np.stack(test_df['distilbert_embeddings']), dtype=torch.float32).to(device)
test_play_counts = torch.tensor(test_df['play_count_scaled'].values, dtype=torch.float32).to(device)

# Step 7: Create a simple edge index (self-loops, i.e., each node connects to itself)
num_nodes_train = train_embeddings.shape[0]
edge_index_train = torch.stack([torch.arange(num_nodes_train), torch.arange(num_nodes_train)], dim=0).to(device)

num_nodes_test = test_embeddings.shape[0]
edge_index_test = torch.stack([torch.arange(num_nodes_test), torch.arange(num_nodes_test)], dim=0).to(device)

# Step 8: Define the GCN model
class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.dropout(x)
        x = self.conv4(x, edge_index)
        return x

# Step 9: Initialize the model with the provided hyperparameters
learning_rate = 8.032781689976049e-05
hidden_size = 128
num_epochs = 11
dropout = 0.3

model = GCNModel(in_channels=train_embeddings.shape[1], hidden_channels=hidden_size, out_channels=1, dropout=dropout)
model = model.to(device)

optimizer = Adam(model.parameters(), lr=learning_rate)

# Step 10: Define training loop
def train_model(model, optimizer, num_epochs, train_embeddings, train_play_counts, edge_index_train):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        out = model(train_embeddings, edge_index_train)  # Forward pass with edge index
        loss = F.mse_loss(out.squeeze(), train_play_counts)  # Calculate loss (MSE)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update parameters

        # Print the training loss for each epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Step 11: Train the model on the training set
train_model(model, optimizer, num_epochs, train_embeddings, train_play_counts, edge_index_train)

# Step 12: Evaluate the model on the test set
model.eval()
with torch.no_grad():
    predicted_test_play_counts = model(test_embeddings, edge_index_test).cpu().numpy()

    # Re-transform predicted play counts back to original scale
    predicted_test_play_counts = np.expm1(scaler_play_count.inverse_transform(predicted_test_play_counts.reshape(-1, 1)))

    # Calculate MAE and RMSE on the test set
    actual_test_play_counts = test_df['play_count'].values
    mae_test = mean_absolute_error(actual_test_play_counts, predicted_test_play_counts)
    rmse_test = np.sqrt(mean_squared_error(actual_test_play_counts, predicted_test_play_counts))

    print(f"Test MAE: {mae_test}")
    print(f"Test RMSE: {rmse_test}")

# Step 13: Make predictions for specific user-song pairs using the test set embeddings
def predict_for_user_song(user_id, song_id, df, embeddings, edge_index):
    try:
        # Find the index of the user-song pair in the DataFrame
        user_song_row = df[(df['user_id'] == user_id) & (df['song_id'] == song_id)]
        if user_song_row.empty:
            print(f"No data found for user {user_id} and song {song_id}.")
            return None

        # Extract the corresponding embedding
        song_embedding = torch.tensor(np.stack(user_song_row['distilbert_embeddings']), dtype=torch.float32).to(device)

        # Make prediction using the model
        model.eval()
        with torch.no_grad():
            predicted_play_count = model(song_embedding, edge_index).cpu().numpy()
            predicted_play_count = np.expm1(scaler_play_count.inverse_transform(predicted_play_count.reshape(-1, 1)))

        # Return the predicted play count
        return predicted_play_count.item()

    except Exception as e:
        print(f"Error predicting for user {user_id} and song {song_id}: {str(e)}")
        return None

# Example usage with test data:
pred_22570_1286 = predict_for_user_song(22570, 1286, test_df, test_embeddings, edge_index_test)
print(f"Predicted play count for user 22570 and song 1286: {pred_22570_1286}")

pred_63855_1286 = predict_for_user_song(63855, 1286, test_df, test_embeddings, edge_index_test)
print(f"Predicted play count for user 63855 and song 1286: {pred_63855_1286}")

# Save model to google drive as a pickle file
model_path = '/content/drive/MyDrive/Python Course/model.pkl'

with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print(f'Model saved to {model_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Epoch 1/11, Loss: 0.18076615035533905
Epoch 2/11, Loss: 0.1633910834789276
Epoch 3/11, Loss: 0.15086066722869873
Epoch 4/11, Loss: 0.1424810141324997
Epoch 5/11, Loss: 0.13652963936328888
Epoch 6/11, Loss: 0.1321597397327423
Epoch 7/11, Loss: 0.12752041220664978
Epoch 8/11, Loss: 0.12321554124355316
Epoch 9/11, Loss: 0.11841163039207458
Epoch 10/11, Loss: 0.11339209228754044
Epoch 11/11, Loss: 0.10782990604639053
Test MAE: 5.980533564805937
Test RMSE: 9.982094514494719
Predicted play count for user 22570 and song 1286: 5.109489917755127
No data found for user 63855 and song 1286.
Predicted play count for user 63855 and song 1286: None
Model saved to /content/drive/MyDrive/Python Course/model.pkl


create a recommendation system

In [ ]:
import pickle
import torch
import numpy as np
import pandas as pd

# Step 1: Mount Google Drive to access the model and dataset
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Load the saved GNN model
model_path = '/content/drive/MyDrive/Python Course/model.pkl'
with open(model_path, 'rb') as file:
    gnn_model = pickle.load(file)

# Step 3: Load the df_with_embeddings pickle file
df_embeddings_path = '/content/drive/MyDrive/Python Course/df_with_embeddings.pkl'
with open(df_embeddings_path, 'rb') as file:
    df_with_embeddings = pickle.load(file)

# Set device to GPU (or CPU if GPU is unavailable)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Ensure the model is in evaluation mode
gnn_model.eval()
gnn_model = gnn_model.to(device)

# Apply log transformation and scaling (ensure the scaler is same as used during training)
scaler_play_count = MinMaxScaler()
df_with_embeddings['play_count_log'] = np.log1p(df_with_embeddings['play_count'])
df_with_embeddings['play_count_scaled'] = scaler_play_count.fit_transform(df_with_embeddings[['play_count_log']])

# Step 4: Define the sample user ID
sample_user_id = 22570

# Step 5: Filter out songs that the user has already listened to
# Get the songs user 22570 has listened to
user_songs = df_with_embeddings[df_with_embeddings['user_id'] == sample_user_id]

# Songs the user has already listened to
listened_songs = user_songs[['song_id', 'title', 'artist_name', 'play_count']]

# Get all songs excluding the ones the user has already played
unplayed_songs = df_with_embeddings[~df_with_embeddings['song_id'].isin(user_songs['song_id'])]

# Step 6: Remove duplicates from the unplayed songs (based on song_id, or use other columns as needed)
unplayed_songs = unplayed_songs.drop_duplicates(subset='song_id')

# Step 7: Function to predict play count using the GNN model for recommendations
def predict_play_count(model, song_row):
    # Extract embeddings and move to the correct device
    embeddings = torch.tensor(np.stack(song_row['distilbert_embeddings']), dtype=torch.float32).unsqueeze(0).to(device)

    # Create an edge index for self-loop (since each song is treated as its own node)
    edge_index = torch.tensor([[0], [0]], dtype=torch.long).to(device)

    # Forward pass through the GNN model
    with torch.no_grad():
        predicted_play_count = model(embeddings, edge_index)  # Use edge_index like in training
        predicted_play_count = predicted_play_count.cpu().numpy()

    # Re-transform predicted play count back to original scale
    predicted_play_count = np.expm1(scaler_play_count.inverse_transform(predicted_play_count.reshape(-1, 1)))

    return predicted_play_count.item()

# Step 8: Make predictions for the unplayed songs
predictions = []
for _, song_row in unplayed_songs.iterrows():
    predicted_count = predict_play_count(gnn_model, song_row)
    predictions.append((song_row['song_id'], song_row['title'], song_row['artist_name'], predicted_count))

# Step 9: Convert predictions to a DataFrame
predicted_df = pd.DataFrame(predictions, columns=['song_id', 'title', 'artist_name', 'predicted_play_count'])

# Step 10: Rank the songs by predicted play count and recommend the top 5
top_5_recommendations = predicted_df.sort_values(by='predicted_play_count', ascending=False).head(5)

# Step 11: Get the top 5 songs user 22570 has already played
top_5_listened_songs = listened_songs.sort_values(by='play_count', ascending=False).head(5)

# Step 12: Display the results
print("Top 5 Recommended Songs for User 22570:")
print(top_5_recommendations[['song_id', 'title', 'artist_name', 'predicted_play_count']])

print("\nTop 5 Songs User 22570 Has Already Played:")
print(top_5_listened_songs[['song_id', 'title', 'artist_name', 'play_count']])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Top 5 Recommended Songs for User 22570:
      song_id                                       title  \
143      6413                             Kaaos ei karkaa   
7591      238                             TE QUIERO PUTA!   
6662     7235                                        Rose   
7645     3123                               Luz Dos Olhos   
4429     1081  Acende O Candieiro (2003 Digital Remaster)   

                             artist_name  predicted_play_count  
143                      Haloo Helsinki!             45.885681  
7591                           Rammstein             37.988350  
6662                    A Perfect Circle             33.581852  
7645                          Nando Reis             33.338898  
4429  Adoniran Barbosa - Com Nosso Samba             33.188675  

Top 5 Songs User 22570 Has Already Played:
    song_id